Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# Создаем датафрейм со средним временем жизни пользователей
user_id = []
user_av_time = []

for i in range(ratings.userId.max()):
    user_n = i + 1
    v_df = ratings.loc[ratings.userId == user_n]
    av_time = v_df.timestamp.max() - v_df.timestamp.min()
    user_id.append(user_n)
    user_av_time.append(av_time)
    
user_time_life = pd.DataFrame(
    {
        'userId': user_id,
        'avTime': user_av_time
    }
)
user_time_life.head()

,userId,avTime
0,1,739163
1,2,505
2,3,970
3,4,62496114
4,5,590


In [7]:
# группируем по userId для расчета количества оценок
ratings = ratings.groupby('userId').agg({'rating': 'sum'})

In [8]:
ratings.head()

,rating
userId,
1,1013.0
2,114.5
3,95.0
4,768.0
5,160.0


In [9]:
len(ratings)

610

In [10]:
# фильтруем с количеством оценок более 100
ratings_sto = ratings.loc[ratings.rating >100].reset_index()
ratings_sto.head()

,userId,rating
0,1,1013.0
1,2,114.5
2,4,768.0
3,5,160.0
4,6,1097.0


In [11]:
len(ratings_sto)

522

In [12]:
len(user_time_life)

610

In [15]:
# объединяем таблицы
ratings_sto.merge(user_time_life, on = 'userId').head()

,userId,rating,avTime
0,1,1013.0,739163
1,2,114.5,505
2,4,768.0,62496114
3,5,160.0,590
4,6,1097.0,3806


In [16]:
# потерянных или лишних строк нет
len(ratings_sto)

522

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы: таблицу с тремя типами выручки для каждого client_id без указания адреса клиента аналогичную таблицу по типам выручки с указанием адреса клиента Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [17]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [25]:
len(rzd)

5

In [18]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [26]:
len(auto)

5

In [19]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [27]:
len(air)

4

In [20]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [28]:
len(client_base)

8

In [31]:
# создаем таблицу по всем перевозкам без адресов клиетов
traffic_revenue = rzd.merge(auto, how = 'outer', on ='client_id').merge(air, how = 'outer', on ='client_id')

In [32]:
traffic_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [33]:
# Проверяем на потери через суммирование соответсвующих столбцов
traffic_revenue.rzd_revenue.sum() == rzd.rzd_revenue.sum()

True

In [34]:
traffic_revenue.auto_revenue.sum() == auto.auto_revenue.sum()

True

In [35]:
traffic_revenue.air_revenue.sum() == air.air_revenue.sum()

True

In [36]:
traffic_adr = traffic_revenue.merge(client_base, how = 'outer', on ='client_id')

In [37]:
traffic_adr

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


In [38]:
# Проверяем на потери через длину таблиц
len(traffic_revenue) ==len(traffic_adr)

True

Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.

1. Как бы вы добавили эти сведения в таблицу визитов и покупок?

   Для составления ответа можно использовать вопросы:
   - У каждого пользователя известен набор координат.
   - А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа.
     Как их получить?

2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. добавил бы через user_id методом merge
    **"А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?"**
    преобразовать в десятичные доли градуса
2. в расчеты попадут первые встретившиеся координаты.
3. регион, из координат мы вычислим регион и составим массовую долю потребности в товаре, логистические маршруты и т.д.
   